In [1]:
# Model Training
from datasets import load_from_disk

datasets = load_from_disk("./encoded_data4")
datasets

c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', 'Threats and Security on the Internet', '__index_level_0__'],
        num_rows: 644
    })
    valid: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', '

In [2]:
datasets = datasets.remove_columns(['__index_level_0__'])

In [3]:
datasets

DatasetDict({
    train: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', 'Threats and Security on the Internet'],
        num_rows: 644
    })
    valid: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', 'Threats and Security 

In [4]:
model_labels = [label for label in datasets['train'].features.keys() if label not in ['Unnamed: 0', 'String', '__index_level_0__']]

In [5]:
model_labels

['Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet']

In [6]:
datasets["train"].column_names

['String',
 'Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet']

In [7]:
datasets["train"].features

{'String': Value(dtype='string', id=None),
 'Algorithm Design': Value(dtype='int32', id=None),
 'Basic Machine Organisation': Value(dtype='int32', id=None),
 'Computer System': Value(dtype='int32', id=None),
 'Data Manipulation and Analysis': Value(dtype='int32', id=None),
 'Data Organisation and Data Control': Value(dtype='int32', id=None),
 'Elementary Web Authoring': Value(dtype='int32', id=None),
 'Health and Ethical Issues': Value(dtype='int32', id=None),
 'Information Processing': Value(dtype='int32', id=None),
 'Intellectual Property': Value(dtype='int32', id=None),
 'Internet Services and Applications': Value(dtype='int32', id=None),
 'Multimedia Elements': Value(dtype='int32', id=None),
 'Networking and Internet Basics': Value(dtype='int32', id=None),
 'Program Development': Value(dtype='int32', id=None),
 'Spreadsheets and Databases': Value(dtype='int32', id=None),
 'Threats and Security on the Internet': Value(dtype='int32', id=None)}

In [8]:
#print(f"{datasets.train.column_names}\n{datasets.train.features}")

In [9]:
model_labels

['Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet']

In [10]:
id2label = {idx:label for idx, label in enumerate(model_labels)}
label2id = {label:idx for idx, label in enumerate(model_labels)}

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [12]:
def encode_data(dataset):
    text = dataset["String"]
    # tokenize string
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=70)
    # create encoded array list with labels
    labelsBatch = {x: dataset[x] for x in dataset.keys() if x in model_labels}
    labels_matrix = np.zeros((len(text), len(model_labels)))
    for idx, label in enumerate(model_labels):
        labels_matrix[:, idx] = labelsBatch[label]
    
    encoding["labels"] = labels_matrix.tolist()
    return encoding

In [13]:
# tokenize the dataset
encoded_ds = datasets.map(encode_data, batched=True, remove_columns=datasets['train'].column_names)

Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data4\train\cache-83276ad8f96c0d70.arrow
100%|██████████| 1/1 [00:00<00:00, 47.77ba/s]
Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data4\test\cache-6fff698335c533a9.arrow


In [14]:
encoded_ds['train']['labels'][0]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]

In [15]:
[id2label[idx] for idx, label in enumerate(encoded_ds['train'][15]['labels']) if label == 1.0]

['Data Organisation and Data Control']

In [16]:
import torch
from torch import cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

encoded_ds.set_format("torch")

Using device: cuda



In [17]:
encoded_ds["train"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 644
})

In [18]:

# define model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", problem_type="multi_label_classification", num_labels=len(model_labels), id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [19]:
# train model
batch_size = 8
metric_name = "f1"

In [20]:
import evaluate
metric  = evaluate.load("accuracy")

In [21]:
import time

train_name_dir = time.strftime(f"%Y%m%d_%H%M%S$-train-model-{model.name_or_path}")
train_name_dir

'20230329_065154$-train-model-bert-base-uncased'

In [22]:
from transformers import TrainingArguments, Trainer


args = TrainingArguments(
    output_dir="./Bert-v1/"+ train_name_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= 81,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='./Bert-v1/logging/'+ train_name_dir
)

In [23]:
args.learning_rate

2e-05

In [24]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)

c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, balanced_accuracy_score, hamming_loss
from transformers import EvalPrediction


def multi_label_metrics(predictions, labels, threshold=0.50):
    # apply sigmoid on predictions fitting (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    # logits, labels = predictions
    # prediction = np.argmax(logits, axis=-1)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'balanced_accuracy': balanced_accuracy,
               }
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

def compute_metrics_new(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [26]:
encoded_ds['train'][0]['labels'].type()

'torch.FloatTensor'

In [27]:
# model.to(device)

In [28]:
outputs = model(input_ids=encoded_ds['train']['input_ids'][0].unsqueeze(0), labels=encoded_ds['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7149, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.8069,  0.2389, -0.1406, -0.2847,  0.1737, -0.0554,  0.0414, -0.0071,
         -0.7592, -0.3400, -0.2981,  0.4890, -0.1173, -0.0797, -0.0747]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["valid"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [30]:
trainer.train()

***** Running training *****
  Num examples = 644
  Num Epochs = 81
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6561
  Number of trainable parameters = 109493775
  1%|          | 80/6561 [00:08<09:22, 11.53it/s] ***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                 
  1%|          | 81/6561 [00:08<09:22, 11.53it/s]Saving model checkpoint to ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-81
Configuration saved in ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-81\config.json


{'eval_loss': 0.2676461637020111, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_balanced_accuracy': 0.06666666666666667, 'eval_runtime': 0.2199, 'eval_samples_per_second': 368.27, 'eval_steps_per_second': 50.012, 'epoch': 1.0}


Model weights saved in ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-81\pytorch_model.bin
tokenizer config file saved in ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-81\tokenizer_config.json
Special tokens file saved in ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-81\special_tokens_map.json
  2%|▏         | 162/6561 [00:17<08:48, 12.10it/s]***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                  
  2%|▏         | 162/6561 [00:17<08:48, 12.10it/s]Saving model checkpoint to ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-162
Configuration saved in ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-162\config.json


{'eval_loss': 0.24016331136226654, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_balanced_accuracy': 0.06666666666666667, 'eval_runtime': 0.2229, 'eval_samples_per_second': 363.326, 'eval_steps_per_second': 49.341, 'epoch': 2.0}


Model weights saved in ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-162\pytorch_model.bin
tokenizer config file saved in ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-162\tokenizer_config.json
Special tokens file saved in ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-162\special_tokens_map.json
  4%|▎         | 243/6561 [00:26<08:39, 12.15it/s]***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                  
  4%|▎         | 243/6561 [00:26<08:39, 12.15it/s]Saving model checkpoint to ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-243
Configuration saved in ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-243\config.json


...

100%|██████████| 6561/6561 [24:25<00:00, 11.97it/s]***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                   
100%|██████████| 6561/6561 [24:25<00:00, 11.97it/s]Saving model checkpoint to ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-6561
Configuration saved in ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-6561\config.json


{'eval_loss': 0.09601752460002899, 'eval_f1': 0.8024691358024691, 'eval_roc_auc': 0.8730354089851072, 'eval_accuracy': 0.7530864197530864, 'eval_balanced_accuracy': 0.7786724386724388, 'eval_runtime': 0.225, 'eval_samples_per_second': 360.036, 'eval_steps_per_second': 48.894, 'epoch': 81.0}


Model weights saved in ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-6561\pytorch_model.bin
tokenizer config file saved in ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-6561\tokenizer_config.json
Special tokens file saved in ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-6561\special_tokens_map.json
100%|██████████| 6561/6561 [24:39<00:00, 11.97it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./Bert-v1/20230329_065154$-train-model-bert-base-uncased\checkpoint-3402 (score: 0.825).
100%|██████████| 6561/6561 [24:41<00:00,  4.43it/s]

{'train_runtime': 1481.9002, 'train_samples_per_second': 35.201, 'train_steps_per_second': 4.427, 'train_loss': 0.030787286900961622, 'epoch': 81.0}


TrainOutput(global_step=6561, training_loss=0.030787286900961622, metrics={'train_runtime': 1481.9002, 'train_samples_per_second': 35.201, 'train_steps_per_second': 4.427, 'train_loss': 0.030787286900961622, 'epoch': 81.0})

In [31]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
100%|██████████| 11/11 [00:00<00:00, 42.85it/s]


{'eval_loss': 0.08496356755495071,
 'eval_f1': 0.825,
 'eval_roc_auc': 0.8801779718623189,
 'eval_accuracy': 0.7654320987654321,
 'eval_balanced_accuracy': 0.7993073593073593,
 'eval_runtime': 0.4151,
 'eval_samples_per_second': 195.146,
 'eval_steps_per_second': 26.501,
 'epoch': 81.0}

In [32]:
prediction_result = trainer.predict(encoded_ds['test'])

***** Running Prediction *****
  Num examples = 81
  Batch size = 8
100%|██████████| 11/11 [00:00<00:00, 61.92it/s]


In [33]:
print(prediction_result.metrics)

{'test_loss': 0.12187018245458603, 'test_f1': 0.7625, 'test_roc_auc': 0.860402741708885, 'test_accuracy': 0.7283950617283951, 'test_balanced_accuracy': 0.7782251082251082, 'test_runtime': 0.1996, 'test_samples_per_second': 405.841, 'test_steps_per_second': 55.114}


In [34]:
trainer.save_model("./Bert-model-v")

Saving model checkpoint to ./Bert-model-v
Configuration saved in ./Bert-model-v\config.json
Model weights saved in ./Bert-model-v\pytorch_model.bin
tokenizer config file saved in ./Bert-model-v\tokenizer_config.json
Special tokens file saved in ./Bert-model-v\special_tokens_map.json


In [35]:
tokenizer.save_pretrained("./Bert-model-v1/tokenizer")

tokenizer config file saved in ./Bert-model-v1/tokenizer\tokenizer_config.json
Special tokens file saved in ./Bert-model-v1/tokenizer\special_tokens_map.json


('./Bert-model-v1/tokenizer\\tokenizer_config.json',
 './Bert-model-v1/tokenizer\\special_tokens_map.json',
 './Bert-model-v1/tokenizer\\vocab.txt',
 './Bert-model-v1/tokenizer\\added_tokens.json',
 './Bert-model-v1/tokenizer\\tokenizer.json')